In [2]:
from bs4 import BeautifulSoup
import requests
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [11]:
texts = []

In [12]:
url = "https://www.sante.gov.ma/sites/Ar/Pages/activites.aspx"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.content, "html.parser")
h3_elements = soup.select("#ctl00_ctl42_g_5c5a1f3c_eda6_4491_a25d_453a88e05f81 h3")
for h3 in h3_elements:
    texts.append(h3.text.strip())

In [13]:
print(texts)

['الأنشطة', 'وزير الصحة والحماية الاجتماعية يجري مباحثات مع مسؤولين عن المعهد الهندي للأمصال', 'وزير الصحة والحماية الاجتماعية يستعرض في ندوة وطنية معالم إصلاح المنظومة الصحية الوطنية بين الماضي والحاضر وتحديات المستقبل', 'تعزيز التعاون في مجال الصحة والحماية الاجتماعية محور مباحثات بين وزير الصحة والحماية الاجتماعية والمدير العام لمنظمة العمل الدولية', 'إعطاء انطلاقة خدمات 15 مؤسسة صحية مع وحدة المستعجلات الطبية على مستوى جهة العيون الساقية الحمراء', 'إعطاء انطلاقة خدمات   14 مركزا للرعاية الصحية الأولية على مستوى جهة فاس- مكناس', 'تعزيز التعاون في مجال الصحة محور مباحثات بين وزير الصحة والحماية الاجتماعية ونظيره التشيكي', 'الحملة الوطنية للتشجيع على نمط العيش السليم "اختاروا الأفضل لصحتكم"', 'المصادقة على أول قانون يتعلق بمزاولة مهن التمريض', 'تخليد اليوم العالمي لمحاربة داء السل', 'الاحتفال باليوم العالمي للمهاجر']


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

driver.get('https://www.who.int/ar/news')

def extract_headings():
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p.heading.text-underline'))
    )
    headings_elements = driver.find_elements(By.CSS_SELECTOR, 'p.heading.text-underline')
    return [heading.text for heading in headings_elements]

all_headings = []
all_headings.extend(extract_headings())

def go_to_next_page():
    next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]'))
    )
    driver.execute_script("arguments[0].click();", next_button)

while True:
    try:
        go_to_next_page()
        WebDriverWait(driver, 10).until(
            EC.staleness_of(driver.find_element(By.CSS_SELECTOR, 'p.heading.text-underline'))
        )
        new_headings = extract_headings()
        if not new_headings:
            break
        all_headings.extend(new_headings)
    except Exception as e:
        print(f"Exception occurred: {e}")
        break

driver.quit()

In [ ]:
data = pd.DataFrame(all_headings + texts, columns = ['Texts'])
data.to_csv('scraping.csv', index = False)

In [41]:
data = pd.read_csv('scraping.csv')
data

,Texts
0,"شعار ""الجميع من أجل الصحة، والصحة من أجل الجمي..."
1,تقرير جديد يشير إلى زيادة كبرى في عدوى الأمراض...
2,اعتماد أكبر عدد من الوكالات التنظيمية للمنتجات...
3,إحراز تقدم غير مسبوق في مفاوضات الدول الأعضاء ...
4,المنظّمة تحدّث قائمة أنواع البكتيريا المقاومة ...
...,...
1193,تعزيز التعاون في مجال الصحة محور مباحثات بين و...
1194,"الحملة الوطنية للتشجيع على نمط العيش السليم ""ا..."
1195,المصادقة على أول قانون يتعلق بمزاولة مهن التمريض
1196,تخليد اليوم العالمي لمحاربة داء السل


In [44]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import nltk
from nltk.corpus import stopwords

In [45]:
arabic_stop_words = list(stopwords.words('arabic'))

def preprocess_texts(texts):
    return texts.str.lower().str.replace(r'\W', ' ', regex=True)

In [46]:
data['Texts'] = preprocess_texts(data['Texts'])
vectorizer = CountVectorizer(stop_words = arabic_stop_words)
dtm = vectorizer.fit_transform(data['Texts'])

In [47]:
n_topics = 5  
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda.fit(dtm)

topic_distribution = lda.transform(dtm)

In [48]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

print_top_words(lda, vectorizer.get_feature_names_out(), 5)

Topic #0:
بشأن الصحة العالمية 19 بيان
Topic #1:
الصحة العالمية بشأن منظمة رقم
Topic #2:
الصحة العالمية منظمة بشأن العالمي
Topic #3:
الصحة العالمية العالمي 19 كوفيد
Topic #4:
الصحة العالمية منظمة 19 كوفيد



In [49]:
health_topic_index = 4

def calculate_scores(topic_distribution, topic_index):
    scores = topic_distribution[:, topic_index] * 10
    return scores

data['Scores'] = calculate_scores(topic_distribution, health_topic_index)

In [71]:
threshold = 0.1  
data = data[data['Scores'] > threshold]

In [3]:
data = pd.read_csv("scrapingScore.csv")
data

,Texts,Scores
0,شعار الجميع من أجل الصحة والصحة من أجل الجمي...,0.157534
1,اعتماد أكبر عدد من الوكالات التنظيمية للمنتجات...,9.453587
2,إحراز تقدم غير مسبوق في مفاوضات الدول الأعضاء ...,0.168359
3,المنظ مة تحد ث قائمة أنواع البكتيريا المقاومة ...,0.156182
4,المنظ مة تختبر مسبقا صلاحية لقاح جديد لحم ى ا...,0.204761
...,...,...
1125,تعزيز التعاون في مجال الصحة محور مباحثات بين و...,9.380450
1126,الحملة الوطنية للتشجيع على نمط العيش السليم ا...,0.201207
1127,المصادقة على أول قانون يتعلق بمزاولة مهن التمريض,0.289254
1128,تخليد اليوم العالمي لمحاربة داء السل,0.286693


In [4]:
import pandas as pd
from farasa.stemmer import FarasaStemmer
import re

farasa_stemmer = FarasaStemmer()

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = text.lower()
    tokens = text.split()
    tokens = [token for token in tokens if token not in arabic_stop_words]
    stemmed_tokens = farasa_stemmer.stem(tokens)
    return stemmed_tokens

example_text = "هذا نص تجريبي لاختبار تطبيق أنابيك، لمعالجة وتحليل النصوص العربية."

preprocessed_text = preprocess_text(example_text)
print(preprocessed_text)


  0%|          | 0.00/241M [00:00<?, ?iB/s]